In [325]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
from re import sub

df = pd.read_csv('[WeVis] They Work for Us - Politician Data - [T] PeopleVote.csv', skiprows=1)
df = df.iloc[:, :9]
df_ocr = pd.read_csv('20220209190321A17.csv', skiprows=1)

In [326]:
df_ocr.columns = ['no', 'id', 'fullname', 'party', 'vote']
df_ocr = df_ocr.astype(np.str_)

In [327]:
def similar(a, b):
    return SequenceMatcher(lambda x: x in ' ', a, b).ratio()

In [328]:
def fix_party(p, ps):
  if not p: return p

  for x in ps:
    sim = similar(p, str(x))
    if sim > .8:
      return x
  return p

df_ocr.party = df_ocr.party.apply(lambda x: fix_party(x.replace('พรรศ', 'พรรค').replace('พรรค', ''), df.party.unique()))

In [329]:
party_b = df_ocr.party.isin(df.party.dropna())
party_b

0      False
1      False
2      False
3      False
4      False
       ...  
507     True
508     True
509    False
510    False
511    False
Name: party, Length: 512, dtype: bool

In [330]:
def split_name(name, titles):
  name = sub('[!-~]', '', name)
  for t in titles:
    if name.startswith(t):
      return name[len(t):]
  return name

df_ocr.fullname = df_ocr.fullname.apply(lambda x: split_name(x, df.title.unique()))

(array([  6,  11,  13,  15,  16,  17,  18,  19,  20,  22,  24,  26,  29,
         35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  48,
         49,  50,  51,  52,  53,  54,  56,  58,  59,  61,  62,  63,  65,
         68,  69,  70,  71,  73,  74,  75,  77,  79,  81,  82,  84,  88,
         89,  90,  92,  95,  96,  97,  98, 100, 103, 104, 105, 106, 107,
        109, 110, 112, 113, 115, 117, 118, 119, 122, 126, 127, 128, 129,
        132, 133, 134, 135, 138, 140, 142, 143, 146, 147, 149, 153, 154,
        155, 156, 158, 159, 160, 164, 165, 167, 169, 170, 177, 179, 180,
        181, 186, 187, 188, 191, 195, 196, 197, 199, 203, 205, 209, 210,
        211, 212, 214, 215, 216, 217, 218, 219, 221, 222, 223, 224, 225,
        228, 229, 230, 232, 233, 236, 238, 239, 240, 241, 244, 246, 252,
        254, 255, 258, 261, 263, 264, 265, 267, 268, 269, 273, 274, 275,
        276, 279, 280, 281, 282, 283, 284, 285, 286, 289, 290, 291, 292,
        293, 294, 295, 296, 297, 298, 299, 301, 302

In [338]:
df.name.dropna().__dir__()

['T',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__r

In [350]:
len(df.name.dropna().unique()),len(df.name.dropna())

(873, 1058)

In [360]:
for i, row in df_ocr.iterrows():
  if ' ' not in row.fullname: continue

  name, lastname = row.fullname.split(' ', maxsplit=1)

In [346]:
name_b = df_ocr.fullname.apply(lambda x: x.split(' ')[0]).apply(lambda x: (x == df.name.dropna()).sum())
name_b

0      0
1      0
2      0
3      0
4      0
      ..
507    1
508    0
509    0
510    0
511    0
Name: fullname, Length: 512, dtype: int64

In [ ]:
def last_name(fullname):
  sep_fn = fullname.split(' ', maxsplit=1)
  if (len(sep_fn) > 1):
    return sep_fn[-1].replace('\'', '')
  return fullname

last_name_b = df_ocr.fullname.apply(last_name).isin(df.lastname.dropna())
last_name_b

0      False
1      False
2      False
3      False
4      False
       ...  
507     True
508    False
509    False
510    False
511    False
Name: fullname, Length: 512, dtype: bool

In [ ]:
name_b.sum(),last_name_b.sum(),(name_b|last_name_b).sum(),party_b.sum()

(298, 306, 397, 470)

In [ ]:
df_cleaned = df_ocr[(name_b)|(last_name_b)|(party_b)]
df_cleaned

,no,id,fullname,party,vote
6,.,383,สมบัติ ศรีสุรินทร์,เพื่อไทย,เห็นด้วย
7,2,009,กตฤษฎา ตันเทอดทิตย์,เพื่อไทย,ไม่เห็นด้วย
8,3,022,กิตตี๋ธัญญา วาจาดี,เพื่อไทย,ไม่เห็นด้วย
9,4ง,023,คิตติศักดิ์ คณาสวัสดิ์,เพื่อไทย,ไม่เห็นด้วย
10,5,035,ขนบายโกศสล ปทบมะ,เพื่อไทย,ไม่เห็นด้วย
...,...,...,...,...,...
504,70,2365,บางสาวศีลับมพา เลิศนุวัฒน์,พลเมืองไทย,ไม่เห็นด้วย
505,47],135,บุญญาพร นาตะธนภัทร,พลังชาติไทย,เห็นด้วย
506,412,296,ระวิ มาศฉผูมาตล,พลังธรรมใหม่,เห็นด้วย
507,ๆ73,179,นิคม บุญวิเศษ,พลังปวงชนไทย,nan


In [ ]:
match_row = np.zeros(df.name.shape)

for i, g in df.groupby('party'):
  tmp = df_cleaned[df_cleaned['party'] == i]
  for j, r in tmp.iterrows():
    sim = g.apply(lambda x: similar(r.fullname.replace(' ', ''), x['name'] + x['lastname']), axis=1)
    if sim.max() > .55:
      match_row[g.iloc[sim.argmax()].name] = j
    else:
      print(r.fullname, sim.max(), g.iloc[sim.argmax()]['lastname'])
    # for k, s in g.iterrows():
    #   if similar(r.fullname.replace(' ', ''), s['name'] + s.lastname) > .5:
    #     match_row[k] = j
    #     break

ตถ เหต ระกูล 0.19047619047619047 มาตรศรี
สมัคร ป้องวงษ์ 0.26666666666666666 เที่ยงธรรม
ลินตา เชิตชัย 0.24242424242424243 เตชะเดชเรืองกุล
ศรัณย์วุฒิ ศรัณย์เกตุ 0.0975609756097561 เตชะเดชเรืองกุล
สงคราม กิจเลิศไพโรจน์ 0.14634146341463414 เตชะเดชเรืองกุล


In [ ]:
df.loc[:, 'ocr-name'] = (df_ocr.loc[match_row].fullname).values

In [ ]:
df.sample(10)[['name','lastname','ocr-name']]

,name,lastname,ocr-name
580,ดนัย,มีชูเวท,
679,วราห์,บุญญะสิทธิ์,
845,ธนาคม,จงจิระ,
971,สุริยันต์,สุริยะโชติกุล,
885,พินิจ,พูลเกิด,
640,ผาณิต,นิติทัณฑ์ประภาศ,
1049,เอนก,เหล่าธรรมทัศน์,
796,พิมพ์รพี,พันธุ์วิชาติกุล,พิมพ์รพี พันธุ์วิขาติกุล
469,เสมอกัน,เที่ยงธรรม,เสมอกัน เทียงธรรม
254,พรเพ็ญ,บุญศิริวัฒนกุล,พรเพ็ญ บุญศิริวัฒนกุล


In [ ]:
'''
1 = เห็นด้วย,
2 = ไม่เห็นด้วย,
3 = งดออกเสียง,
4 = ไม่ลงคะแนนเสียง, 
5 = ไม่เข้าร่วมประชุม,
- = ไม่ใช่วาระการประชุม
'''
def vote_encode(vote: str):
  if vote == '-':
    return 5
  if vote == 'เห็นด้วย':
    return 1
  if vote == 'ไม่เห็นด้วย':
    return 2
  if vote == 'งดออกเสียง':
    return 3
  if vote == 'ไม่ลงคะแนนเสียง':
    return 4

  p = (similar(vote,'เห็นด้วย'), similar(vote,'ไม่เห็นด้วย'), similar(vote,'งดออกเสียง'), similar(vote, 'ไม่ลงคะแนนเสียง'))
  i = np.argmax(p)
  if p[i] < .8:
    return np.nan
  return i + 1

In [ ]:
vote_82_ocr = df_ocr.loc[match_row].vote.apply(vote_encode)

In [ ]:
df_ch = pd.read_csv('[WeVis] They Work for Us - Politician Data - [T] PeopleVote.csv', skiprows=1)

In [ ]:
vote_82_ocr.values

array([nan, nan, nan, ...,  1.,  1.,  1.])

In [ ]:
len(df_ch['votelog.__82']),len(vote_82_ocr)

(1058, 1058)

In [ ]:
vote_82_ocr[(df_ch['votelog.__82'].values==vote_82_ocr.apply(str)).values]

Series([], Name: vote, dtype: float64)

In [ ]:
df_ocr.loc[match_row].vote[(df_ch['votelog.__82'].replace('-', 5).astype(np.float16).values==vote_82_ocr.values)==False]

0.0      nan
0.0      nan
0.0      nan
0.0      nan
0.0      nan
        ... 
0.0      nan
0.0      nan
0.0      nan
0.0      nan
416.0      -
Name: vote, Length: 712, dtype: object

In [ ]:
df_ch['votelog.__82'].isna().sum(),vote_82_ocr.isna().sum()

(124, 631)

In [ ]:
(df_ocr.loc[match_row])[df_ch['votelog.__82']=='1'].sample(10)

/var/folders/rk/cgmwyjj17ddfrn5d_vqzdyr00000gn/T/ipykernel_33235/4278817985.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  (df_ocr.loc[match_row])[df_ch['votelog.__82']=='1'].sample(10)


,no,id,fullname,party,vote
290.0,268,ฮี43,สุรศักดิ์ พันธ์เจริญวรกุส,ภูมิใจไทย,เห็นด้วย
373.0,3ๆ7,129,เตขอิศม์ ขาวทอง,ประชาธิปัตย์,เห็นด้วย
462.0,430,108,ซูการ์โน มะทา,ประชาชาติ,งตออกเสียง
130.0,119,397,สรัสบนันท์ อรรณนพพร,เพื่อไทย,-
50.0,43,54,ศรีเรศ โกฎคําลือ,เพื่อไทย,ไม่เห็นด้วย
284.0,262,9373,สนอง เทพอักษรณรงค์,ภูมิใจไทย,เห็นด้วย
384.0,356,209,ประมวล พงศ์ถาวราเตซ,ประชาธิปัตย์,เห็นด้วย
336.0,312,180,นิติพส ผิวเหมาะ,ก้าวไกล,ไม่เห็นด้วย
309.0,287,489,อํานาจ วจิลาวัลย์,ภูมิใจไทย,-
431.0,ศุ01,175,สมศักดิ์ คุณเงิน,เศรษฐกิจไทย,งดออกเสียง
